Registration of Masks

After registering an image, we register it using a specific mask.

In [1]:
%matplotlib inline
import os
from helpers import *

import ants
import SimpleITK as sitk

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

AntsPy version = 0.5.4
SimpleITK version = 2.5.0


In [2]:
BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
print(f'project folder = {BASE_DIR}')

project folder = c:\Users\CHEEZYJEEZY\Desktop\MRIdata_preprocessing


In [3]:
raw_img_path = os.path.join(BASE_DIR, 'assets', 'raw_examples', 'brain-lesion_T1w.nii.gz')
raw_img_ants = ants.image_read(raw_img_path, reorient = 'IAL')

template_img_path = os.path.join(BASE_DIR, 'assets', 'templates', 'mni_icbm152_t1_tal_nlin_sym_09a.nii' )
template_img_ants = ants.image_read(template_img_path, reorient = 'IAL')

mask_img_path = os.path.join(BASE_DIR, 'assets', 'raw_examples', 'brain-lesion_T1w_mask.nii.gz')
mask_img_ants = ants.image_read(mask_img_path, reorient = 'IAL')

explore_3D_array_with_mask_contour(raw_img_ants.numpy(), mask_img_ants.numpy())

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

In [4]:
transformation = ants.registration(
    fixed = template_img_ants,
    moving = raw_img_ants,
    type_of_transform = 'SyN',
    verbose = True
)

antsRegistration -d 3 -r [0000019C9476E988,0000019C9476EE08,1] -m mattes[0000019C9476E988,0000019C9476EE08,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[0000019C9476E988,0000019C9476EE08,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [C:\Users\CHEEZY~1\AppData\Local\Temp\tmpi2wd1gqb,0000019CEF5E34E8,0000019CEF5E3E68] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1


In [5]:
registered_img_ants = transformation['warpedmovout']

In [7]:
raw_examples = [
    'fsl-open-dev_sub-001_T1w.nii.gz',
    'wash-120_sub-001_T1w.nii.gz',
    'kf-panda_sub-01_ses-3T_T1w.nii.gz',
    'listen-task_sub-UTS01_ses-1_T1w.nii.gz'
]

raw_example = raw_examples[0]

In [8]:
out_folder = os.path.join(BASE_DIR, 'assets', 'preprocessed')
out_folder = os.path.join(out_folder, raw_example.split('.')[0])
os.makedirs(out_folder, exist_ok = True)

out_filename = add_suffix_to_filename(raw_example, suffix = 'registered')
out_path = os.path.join(out_folder, out_filename)

print(raw_img_path[len(BASE_DIR) :])
print(out_path[len(BASE_DIR) :])

\assets\raw_examples\brain-lesion_T1w.nii.gz
\assets\preprocessed\fsl-open-dev_sub-001_T1w\fsl-open-dev_sub-001_T1w_registered.nii.gz


In [9]:
registered_img_ants.to_file(out_path)

Move raw mask from native space

In [10]:
registered_mask_img_ants = ants.apply_transforms(
    moving = mask_img_ants,
    fixed = transformation['warpedmovout'],
    transformlist = transformation['fwdtransforms'],
    verbose = True
)

['-d', '3', '-i', '0000019C94D78548', '-o', '0000019C94D79268', '-r', '0000019C94D792C8', '-n', 'linear', '-t', 'C:\\Users\\CHEEZY~1\\AppData\\Local\\Temp\\tmpi2wd1gqb1Warp.nii.gz', '-t', 'C:\\Users\\CHEEZY~1\\AppData\\Local\\Temp\\tmpi2wd1gqb0GenericAffine.mat']


In [ ]:
explore_3D_array_with_mask_contour(
    arr = registered_img_ants.numpy(),
    mask = registered_mask_img_ants.numpy()
)

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

In [13]:
put_filename = add_suffix_to_filename(raw_example, suffix = 'registered')
out_path = os.path.join(out_folder, put_filename)

print(out_path[len(BASE_DIR) :])

\assets\preprocessed\fsl-open-dev_sub-001_T1w\fsl-open-dev_sub-001_T1w_registered.nii.gz


In [14]:
registered_mask_img_ants.to_file(out_path)